## Random Forest Classifier With Pipeline And Hyperparameter Tuning

In [40]:
import seaborn as sns
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [41]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [42]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [43]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [44]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [45]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [46]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [47]:
df['time'] = encoder.fit_transform(df['time'])

In [48]:
df.time.unique()

array([0, 1])

In [50]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [51]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [52]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: time, dtype: int64

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [54]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## Handle Missing Values
from sklearn.preprocessing import StandardScaler  ## Feature scaling
from sklearn.preprocessing import OneHotEncoder  ## Categorical to numerical
from sklearn.compose import ColumnTransformer

In [59]:
categorical_cols = ['sex', 'smoker','day']
numerical_cols = ['total_bill', 'tip','size']


In [60]:
## feature Engineering Automation
## Numerical Pipelines
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), ##Missing Values
        ('scaler',StandardScaler()) ## feature Scaling
    ]

)

#categorical Pipeline
cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')), ## handling Missing values
                ('onehotencoder',OneHotEncoder()) ## Categorical features to numerical
                ]

            )  

In [61]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)

])

In [62]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [ ]:
X_train

array([[-0.33481585,  0.29210141, -0.61218506, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.34596381,  1.78563356, -0.61218506, ...,  0.        ,
         1.        ,  0.        ],
       [-0.45242562, -0.40256471, -0.61218506, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.64209372, -1.44456389, -0.61218506, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.62383924,  0.32683471, -0.61218506, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.71738397, -0.40256471,  0.45476605, ...,  1.        ,
         0.        ,  0.        ]])

In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [70]:
## Model Training Automation
models={
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier()

}

In [71]:
from sklearn.metrics import accuracy_score

In [72]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)

            

        # Predict Testing data
        y_test_pred =model.predict(X_test)

        # Get accuracy for test data prediction
       
        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score
            

            
    return report

In [73]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9672131147540983,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9508196721311475}

In [78]:
classfier=RandomForestClassifier()

In [79]:
## Hypeparameter Tuning
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [80]:
from sklearn.model_selection import RandomizedSearchCV

In [81]:
cv=RandomizedSearchCV(classfier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.946 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.973 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.973 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.917 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.944 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.946 total time=   0.3s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.973 total time=   0.3s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=1.000 total time=   0.3s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.889 total time=   0.3s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.917 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [82]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 5, 'criterion': 'gini'}

## RandomForest Regression Solve it Internal Assignment